In [1]:
# CODE BLOCK #1
# Summary: makes CFG, parent & child dicts, converts to PCFG type
# (def induce) - this makes a pcfg out of a tagged treebank  
# (def record_children) - makes parent and child dicts 
# (def print_probs) - puts rules into a string format so can be made into PCFG

import nltk
import nltk.corpus
nltk.download('treebank')
from nltk.corpus import treebank
from nltk import PCFG

# get count of how many x -> and the different types, to get probability
from collections import Counter
import re

treebank_sentences = treebank.parsed_sents()[0:10] # changed to [0:10] for testing rules/data

# put in each start of parentheses and pop out last one when parentheses close
stack = []
count = 0

dummy = 0
parent_dict = {} # parent: [children]
child_dict = {} # child: [parents]

# called every time a parent is found
def induce(sentence, parent_dict, child_dict):
  # go through char in the sentence looking for '('
  count = 0
  for i in range(0, len(sentence)):
    for char in sentence[i]:
      char = char.lower()
      if sentence[i]=='(':
        if i != 0:
          parent_dict, child_dict = record_children(i, parent_dict, child_dict, sentence)
          count += 1
        else: 
          parent_dict, child_dict = record_children(1, parent_dict, child_dict, sentence)
          count += 1
  return parent_dict, child_dict
    

# function to put children in a dictionary when a parent is found
def record_children(parent_index, parent_dict, child_dict, test):
  # start at where current parent found
  test = test[parent_index-1:]
  test = test.split()
  stack = []
  children = [] # for parent_dict
  # variable for counting parentheses
  count = 0
  # variable for seeing if parent ends with a word, counts any non-terminals in the children
  non_terminal = -1
  # change first digit in range to 1 if TOP before S, otherwise leave as 0
  for i in range(0,len(test)):
    # go through char in each item
    item = test[i]
    for char in item:
      if char=='(':
        # if very start of sentence
        if count==0:
          # need to add dict with children as keys, use for parsing while parent_dict for printing
          parent = item[1:]
        # don't add NONE 
        if parent != '-NONE-':
          if parent not in ",.?!:--``\"\"\'\'":
            if parent == '$':
              # bc reader hates symbols I guess?
              parent = 'dollar_sign'
            parent = re.sub('=', '_equals_', parent) 
            # see if already in parent_dict
            if parent_dict.get(parent) == None:
              parent_dict[parent] = []
              children = []
        # add that non-terminal to stack
        stack.append(item[1:])
        count +=1
        non_terminal += 1
      # at end of a child
      elif char ==')':
      # remove non-terminal from stack, have to do -1 if more than 1 non-terminal in stack
        count -= 1  
        if len(stack) > 1:
          child = stack[-1]
          
           # return parent_dict, child_dict
          stack.remove(child)
          # when converting into pcfg it breaks w/ -NONE-, so removing it for now since seems to represent a missing subject due to movement in the tree
          if child != '-NONE-':
            if child not in ",.?!:--``\"\"\'\'": 
              if child != "\'\'":
                if child_dict.get(child) == None:
                  child_dict[child] = []
              # add parent to child's entry if not already there
                if parent not in child_dict[child]:
                  child_dict[child].append(parent)
                if len(stack) == 1:
                  children.append(child)
        else:
          child = stack[0]
          stack.remove(child)
          if non_terminal == 0:
            # go through next item to get the word:
            word = ''
            for char in test[i]:
              if char!=')':
                # keep proper nouns uppercase and everything else lower to remove duplicate rules
                if parent != "NNP":
                  char = char.lower()
                word += char
            # add word to that symbol in the parent dict
            # handle words like 're that are part of contractions by removing apostrophe, check for possessive apostrophe first
            if word == '\'':
              word = 'pos_apostrophe' # cfg reader doesn't like apostrophes :(
            else:
              word = re.sub('\'', '', word)
            word = '\''+word+'\''
            child = word
            children.append(word)
            if child_dict.get(child) == None:
            # figure out controlling for case in the dict entries
              child_dict[child] = []
            if parent not in child_dict[child]:
              child_dict[child].append(parent)
  
          # make into a tuple so can use Counter later
          children = tuple(children)
          if parent != '-NONE-':
            if parent not in ",.?!:--``\"\"\'\'":
              if children != ():
                parent_dict[parent].append(children)
          return parent_dict, child_dict
      else:
        continue
      if count==0:
        dummy += 1 # do i still use this dummy?

# makes a string with all the rules and their probs to be converted into PCFG
def print_probs(sample_dict):
  final_print = ''
  cfg = ''
  for element in sample_dict:
    # move onto the next element if a parent is punct. or '-NONE-'
    if element in ",.?!":
      continue
    if element == '-NONE-':
      continue
    if element == 'PRP$':
      element = 'PRP-dollar'
    element = re.sub('=', '_equals_', element)
    empty_count = 0
    total = 0
    # go through each of the parent's children
    for item in Counter(sample_dict.get(element)):
      if len(item) == 1:
        if item[0] in ",.?!``":
          continue
        if 'NONE' in item[0]:
          continue
        final_print += str(element) + ' -> ' + str(item[0]) + ' [' + str(Counter(sample_dict.get(element)).get(item)/len(sample_dict.get(element))) + ']\n'
        cfg += str(element) + ' -> ' + str(item[0]) + '\n'
      else:
        s = str(element) + ' -> '
        for tag in item:
          if tag == 'PRP$':
            tag = 'PRP-dollar'
          if tag == '$':
            tag = 'dollar_sign'
          tag = re.sub('=', '_equals_', tag)
          s += str(tag) + ' '
         
        cfg += s + '\n'
        s += '[' + str(Counter(sample_dict.get(element)).get(item)/len(sample_dict.get(element))) + ']\n'

        final_print += s       
  return final_print, cfg

# call our lovely functions :)
for line in treebank_sentences:
  parent_dict, child_dict = induce(str(line), parent_dict, child_dict)

pcfg, cfg = print_probs(parent_dict)
# punctuation doesnt wanna work :(
old_pcfg = PCFG.fromstring(pcfg)


[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


In [2]:
# CODE BLOCK #2
# Summary: converts PCFG to PCNF
# (def remove_hybrids) - removes stuff like (DP -> 'The' NP), but I think this is handled when making the CFG?
# (def remove_unit_productions) - removes stuff like (DP -> 'The' | 'a' | 'an'), again I think this is already handled in CB1?
# (def TBD) - makes all rules only have two items on the rhs, so A -> B C D becomes A -> X1 D and X1 -> B C

import nltk
length = len(old_pcfg.productions())
# STEP 1: change hybrid rules! No word and non-terminal on RHS, like NP -> ('the' N) etc.
def remove_hybrids(grammar, length):
  # list to hold rules that are additional to this grammar, like THE -> 'the'
  extra_rules = []
  # loop through each rule's right side and see if has more than 1 item (terminal or non-terminal)
  for i in range(0,length):
    rule = grammar.productions()[i].rhs()
    rule_len = len(rule)
    if rule_len==1:
      continue
    else:
      word_count = 0
      nt_count = 0
      # loop through each item in right hand of that rule to see if contains both word and non-terminal
      for i in range(0, rule_len):
        # if there's a word/terminal add to respective count
        if 'str' in str(type(rule[i])):
          word_count += 1
        else:
          # it's a non-terminal
          nt_count += 1
        # if there's at least one of each it's a hybrid
        if nt_count > 0 and word_count > 0:
          # list for new rule since can't edit specific items in tuple
          new_rule = []
          # go through each item in righthand side and add to new_rule
          for i in range(0, rule_len):
            # if it's a word, replace w/ uppercase symbol of it instead
            if 'str' in str(type(rule[i])):
              new_rule.append(nltk.grammar.Nonterminal(str(rule[i]).upper()))
              # write THE -> 'the' rule that has to also be added:
              # make list for right side since needs to be in a list
              right_list = []
              right_list.append(str(rule[i]))
              this_rule = [nltk.grammar.Nonterminal(str(rule[i]).upper()), right_list]
              extra_rules.append(tuple(this_rule)) 
            else:
              new_rule.append(rule[i])

          rule = tuple(new_rule)
          # change old rule into new one
          grammar.productions()[i] = nltk.grammar.Production(grammar.productions()[i].lhs(), rule)
          break

  for rule in extra_rules:
    grammar.productions().append(nltk.grammar.Production(rule[0], rule[1]))
  return extra_rules, grammar

# STEP 2 - remove unit productions!
def remove_unit_productions(grammar, length):
  for i in range(0, length):
    rule = grammar.productions()[i].rhs()
    rule_len = len(rule)
    # see if there's 1 item on right side
    if rule_len == 1:
      # see if it's a non-terminal
      if 'str' in str(type(rule[0])):
        continue
      else:
        # handle rules where that unit goes to words
        # find all rules with that unit on left side:
        left_list = grammar.productions(lhs=rule[0])
        #print('HERES LEFT LIST: ', left_list)
        # handle rules where that unit goes to other units (time permitting) # ******LEFT OFF HERE! 

# make rhs 2 or less NTs - still a WIP...

# counter for all the new X1, X2, etc variables
new_var_counter = 0
def two_on_rhs(pcfg, counter):
  #print(toy_pcfg.productions()[5])
  #print('LEN: ', len(toy_pcfg.productions()[5].rhs()))
  for j in range(0, len(pcfg.productions())):
    rule = pcfg.productions()[j]
    if len(rule.rhs()) >2:
      print('rule longer than 2!')
  #rule = pcfg.productions()
      lhs = rule.lhs()
      first_new_rule = 'X' + str(counter) + ' -> ' + str(rule.rhs()[0]) + ' ' + str(rule.rhs()[1])
      # need to check if this rule exists already as something else, will figure out later
      print('FIRST NEW RULE: ', first_new_rule)
      # make rest of the new rules from there
      for i in range(2, len(rule.rhs())):
        # last rule brings back original lhs:
        if i==(len(rule.rhs())-1):
          rule = tuple('X' + str(counter), str(rule.rhs()[i]))
          pcfg.productions()[j] = nltk.grammar.Production(pcfg.productions()[j].lhs(), rule)
          continue
        #print(str(i) + ': ' + str(toy_pcfg.productions()[8].rhs()[i]))
        lhs = 'X' + str(counter+1)
        s = (str(lhs) + ' -> ' + 'X' + str(counter) + ' ' + str(rule.rhs()[i]))
        # add rule to grammar
        pcfg.productions().append(s)
        
        counter += 1


extra_rules, new_pcfg = remove_hybrids(old_pcfg, length)
remove_unit_productions(new_pcfg, length)
#two_on_rhs(new_pcfg, new_var_counter)
print(new_pcfg)

Grammar with 265 productions (start state = S)
    S -> NP-SBJ VP [0.444444]
    S -> NP-SBJ-1 VP [0.0555556]
    S -> NP-SBJ NP-PRD [0.0555556]
    S -> S-TPC-1 NP-SBJ SBAR [0.0555556]
    S -> SBAR-TMP NP-SBJ VP [0.0555556]
    S -> SBAR VP [0.0555556]
    S -> S NP-SBJ PP-LOC [0.0555556]
    S -> NP-SBJ-2 VP [0.0555556]
    S -> NP-SBJ S [0.0555556]
    S -> NP-SBJ SBAR [0.0555556]
    S -> NP-SBJ PP-CLR [0.0555556]
    NP-SBJ -> NP ADJP [0.0625]
    NP-SBJ -> NNP NNP [0.0625]
    NP-SBJ -> PP S-CLR [0.0625]
    NP-SBJ -> NNS [0.125]
    NP-SBJ -> NP NP [0.0625]
    NP-SBJ -> PRP [0.125]
    NP-SBJ -> NP PP [0.0625]
    NP-SBJ -> NP SBAR [0.0625]
    NP-SBJ -> DT JJS NNS [0.0625]
    NP-SBJ -> DT NNP NN [0.0625]
    NP-SBJ -> DT [0.0625]
    NP-SBJ -> NN [0.125]
    NP-SBJ -> EX [0.0625]
    NP -> NNP NNP [0.0701754]
    NP -> CD NNS [0.0350877]
    NP -> DT NN [0.105263]
    NP -> DT JJ NN [0.0526316]
    NP -> NN [0.0701754]
    NP -> NP NP [0.0701754]
    NP -> DT NNP VBG NN [0.0

In [6]:
# CODE BLOCK #3 - makes CKY grid/data structure for storing each cell's POS and backpointers to other cells

import nltk

sample = ['Pierre', 'will', 'join', 'the', 'board']

cky_grid = []
treebank_sentence = (treebank.sents()[0])

print(treebank_sentence)
# make dictionary(instead of dicts inside list/2d array thingy?) for grid-like CKY system, inner lists are columns i guess
# go left to right
def array_maker(sentence):
  cky_grid = {}
  sentence_len = len(sentence)
  print('SENTENCE LEN: ', sentence_len)
  for j in range(sentence_len+1):
    # add a dict for each cell in that column, number it for now
    for i in range(1,len(sentence)+1):
      cky_grid[(j,i)]=[]
  print('HERES THE RESULTING GRID: ', cky_grid)
  return cky_grid, sentence_len

# i by j grid
# go to (i=0,j=0) and get first word's possible POSs from child_dict
# 3) go down to (i+1, j+1) and do the same for second word
# 4) go up to (i=1, j=0) and put possible parents of first and second word, again found from child_dict
# 5) add the backpointers to each parent in (0,1) - FIGURE OUT GOOD DATA STRUCTURE AND/OR MAKE BACKPOINTERS CLASS

# for after 2nd column, goes in pattern: (0,1 & 1,4); (0,2 & 2,4); (0,3 & 3,4) for a column w 4 cells in it

cky_grid, sentence_len = array_maker(sample)

print(cky_grid)


['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.']
SENTENCE LEN:  5
HERES THE RESULTING GRID:  {(0, 1): [], (0, 2): [], (0, 3): [], (0, 4): [], (0, 5): [], (1, 1): [], (1, 2): [], (1, 3): [], (1, 4): [], (1, 5): [], (2, 1): [], (2, 2): [], (2, 3): [], (2, 4): [], (2, 5): [], (3, 1): [], (3, 2): [], (3, 3): [], (3, 4): [], (3, 5): [], (4, 1): [], (4, 2): [], (4, 3): [], (4, 4): [], (4, 5): [], (5, 1): [], (5, 2): [], (5, 3): [], (5, 4): [], (5, 5): []}
{(0, 1): [], (0, 2): [], (0, 3): [], (0, 4): [], (0, 5): [], (1, 1): [], (1, 2): [], (1, 3): [], (1, 4): [], (1, 5): [], (2, 1): [], (2, 2): [], (2, 3): [], (2, 4): [], (2, 5): [], (3, 1): [], (3, 2): [], (3, 3): [], (3, 4): [], (3, 5): [], (4, 1): [], (4, 2): [], (4, 3): [], (4, 4): [], (4, 5): [], (5, 1): [], (5, 2): [], (5, 3): [], (5, 4): [], (5, 5): []}
{(0, 1): [], (0, 2): [], (0, 3): [], (0, 4): [], (0, 5): [], (1, 1): ['MD'], (1, 2): [], (1

In [7]:
# CODE BLOCK 4 - BUILDING UP THE PARSER STEP BY STEP, MOVED BACKPOINTER CLASS HERE!
#   i 0 1 2 3 4
# j
#  1
#  2
#  3
#  4

class Backpointer:
  'objects for storing backpointer info to non-terminal\'s child nodes'
  def __init__(self, name, l, r):
    self.name = name
    self.l = l
    self.r = r
  def display_backpointer(self):
    # string for name of non-terminal, tuple for cell indices ['NP', (0,1)]
    print(self.name)
    print("Left child:  " + str(self.l[0]) + "\tPosition: " + str(self.l[1]))
    print("Right child: " + str(self.r[0]) + "\tPosition: " + str(self.r[1]))

new_backpointer = Backpointer('S', ['NP', (0,0)], ['VP', (1,1)])
new_backpointer.display_backpointer()
sample = ['Pierre', 'join', 'the', 'board']

def new_cky_parser(sentence, cky_grid):
  length = len(sentence)
  # go across the columns 
  for i in range(1, length+1):
    # start from bottom of each column and go up:
    for j in range(i-1, -1, -1):
      #print('j: ',j)
      if j == (i-1):
        current_word = '\'' + str(sentence[i-1]) + '\''
        #print('at bottom of column! POS tag is: ', child_dict[current_word])
        new_backpointer = Backpointer(child_dict[current_word][0], [],[])
        print(cky_grid[(j,i)])
        cky_grid[(j,i)].append(new_backpointer)
      else:
        # start trying out each pivot point, z is number of pivots at that cell:
        pivots = i-j-1
        print(pivots)
        #count = 1
        # get coordinates for both cells in pivot
        for z in range(1, pivots+1):
          print('Pivot #' + str(z) + ': (' + str(j) + ', ' + str(j+z) +') and (' + str(j+z) + ', ' + str(i) +')')
          #count += 1
          first_coordinates = (j, j+z)
          #first_NT = cky_grid[first_coordinates].name

          second_coordinates = (j+z, i)
          #second_NT = cky_grid[second_coordinates].name
          
          # check to see if these cells are empty and quit if they are
          if cky_grid[first_coordinates] == []:
            print('EMPTY 1st NT')
            # skip since no possible lhs
            continue
          # go through each possible NT in that cell
           
          for backpointer in cky_grid[first_coordinates]:
            # get the NT from the cell's backpointer object's 'name'
            first_NT = backpointer.name
            print('FIRST NONTERMINAL: ',first_NT)
            # try each of the second NTs w the first:
            if cky_grid[second_coordinates] == []:
              print('EMPTY 2nd NT')
              # skip since no possible lhs
              continue 
            for backpointer in cky_grid[second_coordinates]:
              second_NT = backpointer.name
            print('SECOND NONTERMINAL: ',second_NT)
            # have to cast into non-terminals then recombine as tuple to check truth
            rule_in_q = (nltk.grammar.Nonterminal(str(first_NT)), nltk.grammar.Nonterminal(str(second_NT)))
            # go through each rule's rhs to find a match:
            for rule in new_pcfg.productions():
              #print(rule.rhs(), 'vs', rule_in_q, '\n')
              if rule.rhs() == rule_in_q:
                print("MATCH FOUND!")
                print('RHS: ', rule.rhs())
                another_new_backpointer = Backpointer(rule.lhs(), [str(first_NT), first_coordinates], [str(second_NT), second_coordinates])
                cky_grid[(j,i)].append(another_new_backpointer)
                
          
  print(cky_grid)
            

          #('Next up, check if these two form a rhs in our grammar!')


new_cky_parser(sample, cky_grid)

S
Left child:  NP	Position: (0, 0)
Right child: VP	Position: (1, 1)
[]
[]
1
Pivot #1: (0, 1) and (1, 2)
FIRST NONTERMINAL:  NNP
SECOND NONTERMINAL:  VB
[]
1
Pivot #1: (1, 2) and (2, 3)
FIRST NONTERMINAL:  VB
SECOND NONTERMINAL:  DT
2
Pivot #1: (0, 1) and (1, 3)
FIRST NONTERMINAL:  NNP
EMPTY 2nd NT
Pivot #2: (0, 2) and (2, 3)
EMPTY 1st NT
[]
1
Pivot #1: (2, 3) and (3, 4)
FIRST NONTERMINAL:  DT
SECOND NONTERMINAL:  NN
MATCH FOUND!
RHS:  (DT, NN)
MATCH FOUND!
RHS:  (DT, NN)
2
Pivot #1: (1, 2) and (2, 4)
FIRST NONTERMINAL:  VB
SECOND NONTERMINAL:  NP-PRD
Pivot #2: (1, 3) and (3, 4)
EMPTY 1st NT
3
Pivot #1: (0, 1) and (1, 4)
FIRST NONTERMINAL:  NNP
EMPTY 2nd NT
Pivot #2: (0, 2) and (2, 4)
EMPTY 1st NT
Pivot #3: (0, 3) and (3, 4)
EMPTY 1st NT
{(0, 1): [<__main__.Backpointer object at 0x7efbea46d610>], (0, 2): [], (0, 3): [], (0, 4): [], (0, 5): [], (1, 1): ['MD'], (1, 2): [<__main__.Backpointer object at 0x7efbea46d410>], (1, 3): [], (1, 4): [], (1, 5): [], (2, 1): [], (2, 2): [], (2, 3): [<

In [ ]:
# CODE BLOCK #5 - Makes rules w >2 NTs in rhs into CNF
from nltk import PCFG
string_cfg = """
N -> \'butterfly\' [0.4]
N -> \'candle\' [0.2]
N -> \'cake\' [0.4]
VP -> \'eats\' [0.3]
VP -> \'runs\' [0.7]
S -> NP VP NNP MD Adv [0.1]
S -> NP VP [0.9]
NP -> \'The\' N [1.0]
Z -> A B C D E F G [1.0]"""
toy_pcfg = PCFG.fromstring(string_cfg)
# counter for all the new X1, X2, etc variables
new_var_counter = 0
print(toy_pcfg.productions()[5])
print('LEN: ', len(toy_pcfg.productions()[5].rhs()))
rule = toy_pcfg.productions()[5]
og_lhs = rule.lhs()
first_new_rule = 'X' + str(new_var_counter) + ' -> ' + str(rule.rhs()[0]) + ' ' + str(rule.rhs()[1])
# need to check if this rule exists already as something else, will figure out if it happens in dev data

print('FIRST NEW RULE: ', first_new_rule)
# make rest of the new rules from there
for i in range(2, len(rule.rhs())):
  if i==(len(rule.rhs())-1):
    print('FINAL NEW RULE: ' + str(og_lhs) + ' -> ' + 'X' + str(new_var_counter) + ' ' + str(rule.rhs()[i]))
    continue
  #print(str(i) + ': ' + str(toy_pcfg.productions()[8].rhs()[i]))
  lhs = 'X' + str(new_var_counter+1)
  print('NEW RULE: ' + str(lhs) + ' -> ' + 'X' + str(new_var_counter) + ' ' + str(rule.rhs()[i]))
  # last rule brings back original lhs:
  new_var_counter += 1
  
  
# Z -> X1 G
# 41 -> X2 F
# X -> X3 E
# X2 -> X4 D
# X1 -> X5 C
# X0 -> A B - will want to check if this rule exists already as another dummy rule, so you don't have a repeat dummy rule! Make this rule first!
#print(sample[-1])
# 

S -> NP VP NNP MD Adv [0.1]
LEN:  5
FIRST NEW RULE:  X0 -> NP VP
NEW RULE: X1 -> X0 NNP
NEW RULE: X2 -> X1 MD
FINAL NEW RULE: S -> X2 Adv
